# Day 11
https://adventofcode.com/2016/day/11

In [3]:
import aocd
data = aocd.get_data(year=2016, day=11)

In [14]:
from collections import defaultdict, deque
from itertools import chain, combinations
import re

In [6]:
chip_descriptions = re.compile(r'(?:a|an) (\w+)-compatible microchip')
gen_descriptions = re.compile(r'(?:a|an) (\w+) generator')

In [16]:
class GameState():
    
    def __init__(self, elevator, items):
        self.elevator = elevator
        items = list(items)
        item_pairs = (items[i:i+2] for i in range(0, len(items), 2))
        self.items = tuple(item for pair in sorted(item_pairs) for item in pair)
    
    def __eq__(self, other):
        return self.elevator == other.elevator and self.items == other.items
    
    def __hash__(self):
        return hash((self.elevator, self.items))
    
    def __repr__(self):
        return 'GameState(elevator={}, items={})'.format(self.elevator, repr(self.items))
    
    def is_legal(self):
        generators = tuple(item for n, item in enumerate(self.items) if n % 2 == 0)
        microchips = tuple(item for n, item in enumerate(self.items) if n % 2 == 1)
        
        for n, microchip in enumerate(microchips):
            if generators[n] != microchip and sum(1 for generator in generators if generator == microchip) > 0:
                return False
        
        return True
    
    def apply_move(self, direction, itemindices):
        return GameState(self.elevator + direction,
                         (item + (direction*int(index in itemindices)) for index, item in enumerate(self.items)))
    
    def all_possible_moves(self):
        items_at_elevator = [itemno for itemno, floor in enumerate(self.items) if floor == self.elevator]
        carry_combos = [(item,) for item in items_at_elevator] + [combo for combo in combinations(items_at_elevator, 2)]
        
        up = down = tuple()
        if self.elevator < 3:
            up = (self.apply_move(+1, combo) for combo in carry_combos)
        if self.elevator > 0:
            down = (self.apply_move(-1, combo) for combo in carry_combos)
        
        return chain(up, down)
    
    @classmethod
    def from_input(cls, input):
        elements = defaultdict(dict)

        for floor, line in enumerate(input.split('\n')):
            for chipdesc in chip_descriptions.finditer(line):
                elements[chipdesc.group(1)]['chip'] = floor
            for gendesc in gen_descriptions.finditer(line):
                elements[gendesc.group(1)]['generator'] = floor

        return cls(0, chain.from_iterable((element['generator'], element['chip'])
                                          for name, element in elements.items()))


In [12]:
def find_shortest_solution(start):
    finish = GameState(3, [3 for item in start.items])
    visited = set()
    search = deque([(0, start)])
    
    while search:
        moves, state = search.popleft()
        
        if state == finish:
            return moves
        elif state not in visited:
            search.extend((moves+1, newstate) for newstate in state.all_possible_moves() if newstate.is_legal())
            visited.add(state)

In [18]:
p1 = find_shortest_solution(GameState.from_input(data))
print('Part 1: {}'.format(p1))
p2 = find_shortest_solution(GameState(0, chain(start.items, (0, 0, 0, 0))))
print('Part 2: {}'.format(p2))

Part 1: 37
Part 2: 61
